# Изградња AI агената са перзистентном меморијом користећи Cognee

Овај нотебук демонстрира како изградити интелигентне AI агенте са напредним способностима меморије користећи [**cognee**](https://www.cognee.ai/) - отворени извор AI меморије који комбинује графове знања, семантичко претраживање и управљање сесијама за креирање AI система свесних контекста.

## 🎯 Циљеви учења

На крају овог туторијала, разумећете како да:
- **Изградите графове знања подржане ембедингима**: Претворите неструктурирани текст у структурисано, упитно знање
- **Имплементирате меморију сесије**: Креирајте разговоре са више корака уз аутоматско задржавање контекста
- **Перзистирате разговоре**: Опционо сачувајте важне интеракције у дугорочној меморији за будућу употребу
- **Постављате упите користећи природни језик**: Приступите и искористите историјски контекст у новим разговорима
- **Визуализујете меморију**: Истражите односе у графу знања вашег агента


## 🏗️ Шта ћете направити

У овом туторијалу, креираћемо **Асистента за кодирање** са перзистентном меморијом који:

### 1. **Конструкција базе знања**
   - Уноси информације о профилу и експертизи програмера
   - Обрађује принципе и најбоље праксе програмирања у Python-у
   - Чува историјске разговоре између програмера и AI асистената

### 2. **Разговори свесни сесије**
   - Одржава контекст кроз више питања у истој сесији
   - Аутоматски кешира сваки пар питање/одговор ради ефикасног преузимања
   - Пружа кохерентне, контекстуалне одговоре засноване на историји разговора

### 3. **Дугорочна меморија**
   - Перзистира важне разговоре у дугорочну меморију
   - Преузима релевантна сећања из базе знања и претходних сесија како би информисао нове интеракције
   - Гради растућу базу знања која се временом побољшава

### 4. **Интелигентно преузимање меморије**
   - Користи семантичко претраживање засновано на графовима за проналажење релевантних информација у целокупној похрањеној бази знања
   - Филтрира претраге по подгрупама података (информације о програмеру vs. принципи)
   - Комбинује више извора података ради пружања свеобухватних одговора


## 📋 Предуслови и подешавање

### Системски захтеви

Пре него што почнете, уверите се да имате:

1. **Python окружење**
   - Python 3.9 или новији
   - Виртуелно окружење (препоручено)
   
2. **Redis кеш** (Потребно за управљање сесијама)
   - Локални Redis: `docker run -d -p 6379:6379 redis`
   - Или користите управљану Redis услугу
   
3. **Приступ LLM API-ју**
   - OpenAI API кључ или други провајдери (погледајте [документацију](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Конфигурација базе података**
   - Подразумевано није потребна конфигурација. Cognee користи базе података засноване на датотекама (LanceDB и Kuzu)
   - Опционално, можете подесити Azure AI Search као складиште вектора (погледајте [документацију](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Конфигурација окружења

Направите `.env` датотеку у вашем пројектном директоријуму са следећим променљивим:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Разумевање архитектуре меморије Cognee

### Како Cognee функционише

Cognee пружа софистициран систем меморије који превазилази једноставно складиштење кључ-вредност:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Кључне компоненте:

1. **Граф знања**: Чува ентитете, односе и семантичке везе
2. **Векторски уграђаји**: Омогућавају семантичку претрагу кроз све сачуване информације
3. **Кеш сесије**: Одржава контекст разговора унутар и између сесија
4. **NodeSets**: Организују податке у логичке категорије за циљано преузимање

### Типови меморије у овом туторијалу:

- **Перзистентна меморија**: Дугорочно складиштење у графу знања
- **Сесијска меморија**: Привремени контекст разговора у Redis кешу
- **Семантичка меморија**: Претраживање сличности засновано на векторима кроз све податке


## 📦 Инсталирајте потребне пакете

Инсталирајте Cognee са Redis подршком за управљање сесијама:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Иницијализуј окружење и учитај библиотеке

Уверите се:
1. Да Redis ради (нпр. преко Docker-а: `docker run -d -p 6379:6379 redis`)
2. Да су променљиве окружења постављене пре него што се увезу модули кеша
3. Ако је потребно, поново покрените језгро и извршите ћелије редом

Следећа ћелија ће:
1. Учитати променљиве окружења из `.env`
2. Конфигурисати Cognee са вашим LLM подешавањима
3. Омогућити кеширање за управљање сесијама
4. Потврдити да су сви компоненти правилно повезани


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Конфигуришите директоријуме за складиштење

Cognee користи два одвојена директоријума за своје операције:
- **Корен података**: Чува унете документе и обрађене податке
- **Системски корен**: Садржи базу података графа знања и системске метаподатке

Креираћемо изоловане директоријуме за овај туторијал на следећи начин:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Ресетовање стања меморије

Пре него што почнемо са изградњом нашег система меморије, хајде да се уверимо да почињемо од почетка.

> 💡 **Савет**: Можете прескочити овај корак ако желите да сачувате постојећа сећања из претходних покретања када касније користите ову свеску.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Део 1: Изградња базе знања

### Извори података за нашег асистента за програмере

Унећемо три врсте података како бисмо креирали свеобухватну базу знања:

1. **Профил програмера**: Лично искуство и техничка позадина
2. **Најбоље праксе за Python**: Зен Python-а са практичним смерницама
3. **Историјски разговори**: Претходне сесије питања и одговора између програмера и AI асистената

Ови разноврсни подаци омогућавају нашем агенту да:
- Разуме технички контекст корисника
- Примени најбоље праксе у препорукама
- Учи из претходних успешних интеракција


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Претварање података у граф знања

Сада ћемо претворити наш необрађени текст у структурирану меморију. Овај процес:

1. **Додаје податке у NodeSets**: Организује информације у логичке категорије
   - `developer_data`: Профил програмера и разговори
   - `principles_data`: Најбоље праксе и смернице за Python

2. **Покреће Cognify Pipeline**: Извлачи ентитете, односе и креира уграђивања
   - Идентификује кључне концепте
   - Ствара семантичке везе између повезаних информација
   - Генерише векторска уграђивања

Ово може потрајати неколико тренутака док LLM обрађује текст и гради структуру графа:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Визуализација графа знања

Хајде да истражимо структуру нашег графа знања. Визуализација приказује:
- **Чворове**: Ентитети извучени из текста (концепти, технологије, људи)
- **Ивице**: Односи и везе између ентитета
- **Кластере**: Повезани концепти груписани по семантичкој сличности

Отворите генерисани HTML фајл у вашем претраживачу да бисте интерактивно истражили граф:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Обогатите меморију са Memify

Функција `memify()` анализира граф знања и генерише интелигентна правила о подацима. Овај процес:
- Идентификује обрасце и најбоље праксе
- Креира применљиве смернице на основу садржаја
- Успоставља односе између различитих области знања

Ова правила помажу агенту да доноси боље информисане одлуке приликом одговарања на питања. Захватање друге визуализације помаже вам да упоредите како се граф густи након обогаћивања.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Део 2: Интелигентно преузимање меморије

### Демонстрација 1: Интеграција знања из више докумената

Сада када је наш граф знања изграђен, тестираћемо како Cognee комбинује информације из више извора да би одговорио на сложена питања.

Први упит демонстрира:
- **Семантичко разумевање**: Проналажење релевантних концепата чак и када нису експлицитно поменути
- **Укрштање информација**: Комбиновање профила програмера са принципима Python-а
- **Контекстуално резоновање**: Примена најбољих пракси на специфичне имплементације

### Демонстрација 2: Филтрирана претрага са NodeSets

Други упит показује како циљати одређене подсетове графа знања:
- Користи параметар `node_name` за претрагу само унутар `principles_data`
- Пружа фокусиране одговоре из специфичне области знања
- Корисно када вам је потребна информација из одређене домене


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Део 3: Подешавање управљања сесијом

### Омогућавање меморије разговора

Управљање сесијом је кључно за одржавање контекста током више интеракција. Овде ћемо:

1. **Иницијализовати кориснички контекст**: Направити или преузети кориснички профил за праћење сесије
2. **Конфигурисати механизам кеша**: Повезати се са Redis-ом за чување историје разговора
3. **Омогућити променљиве сесије**: Поставити контекстуалне променљиве које се задржавају током упита

> ⚠️ **Важно**: Ово захтева да Redis ради и `CACHING=true` у вашем окружењу


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Помоћна функција: Преглед историје сесије

Ова помоћна функција нам омогућава да прегледамо историју разговора сачувану у Redis-у. Корисна је за:
- Отклањање грешака у управљању сесијама
- Проверу да ли се разговори кеширају
- Разумевање контекста који је доступан агенту


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Сесија 1: Лабораторија за асинхрону подршку — Прво питање

Започните сесију `async-support-lab` постављањем питања о телеметријски прилагођеним asyncio обрасцима за масивни веб скрепер. Граф већ познаје asyncio, aiohttp и праксе мониторинга, тако да одговор треба да одражава претходне разговоре док прилагођава одговор новом упиту.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Преглед меморије сесије 1 након прве размене

Покретањем `show_history(session_1)` одмах након почетног питања потврђује се да је Cognee уписао и упит и одговор у Redis. Требало би да видите један унос са смерницама за истовременост.


In [ ]:
await show_history(session_1)

## Сесија 1: Прати праћење модела података

Следеће питање је: "Када треба да изаберем dataclasses уместо Pydantic?" користећи исти ID сесије. Cognee би требало да повеже Python принципе са претходним разговорима о FastAPI-ју како би пружио детаљан савет—показујући да се контекст преноси у оквиру именоване сесије.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Потврдите да историја сесије 1 садржи оба корака

Још један позив `show_history(session_1)` треба да прикаже два уноса питања и одговора. Ово одговара кораку "репродукције меморије" у лабораторији Mem0 и доказује да додатни кораци проширују исти транскрипт.


In [ ]:
await show_history(session_1)

## Сесија 2: Нит за преглед дизајна — Нова сесија

Да бисмо показали изолацију између нити, покрећемо `design-review-session` и тражимо упутства за евидентирање прегледа инцидената. Иако је основна база знања иста, нови идентификатор сесије одржава транскрипте одвојеним.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Преглед Сесије 2 Историја

`show_history(session_2)` треба да приказује само пар питања и одговора из прегледа дизајна. Упоредите то са Сесијом 1 како бисте истакли како Cognee одржава независне транскрипте док истовремено користи заједнички граф знања.


In [ ]:
await show_history(session_2)

## Резиме

Честитамо! Управо сте свом асистенту за кодирање дали прави слој дугорочне меморије који покреће Cognee.

У овом туторијалу сте узели сирови садржај за програмере (код, документацију, разговоре) и претворили га у граф + векторску меморију коју ваш агент може претраживати, анализирати и континуирано унапређивати.

Шта сте научили:

1. **Од сировог текста до AI меморије**: Како Cognee обрађује неструктуриране податке и претвара их у интелигентну, претраживу меморију користећи комбиновану архитектуру вектора + графа знања.

2. **Обогаћивање графа помоћу memify**: Како прећи основно креирање графа и користити memify за додавање изведених чињеница и богатијих односа на постојећи граф.

3. **Вишеструке стратегије претраге**: Како претраживати меморију користећи различите типове претраге (Q&A свесно графа, RAG-стил комплетирање, увиди, сирови делови, претрага кода, итд.) у зависности од потреба вашег агента.

4. **Визуелна експлорација**: Како прегледати и отклонити грешке у ономе што је Cognee изградио користећи визуализације графа и Cognee UI, тако да можете заиста видети како је знање структурирано.

5. **Меморија свесна сесије**: Како комбиновати контекст по сесији са трајном семантичком меморијом тако да агенти могу памтити кроз више покретања без цурења информација између корисника.


## Кључне тачке
1. Меморија као граф знања подржана уграђивањима

    - **Структурисано разумевање**: Cognee комбинује складиште вектора и складиште графова, тако да су ваши подаци претраживи по значењу и повезани односима. Cognee подразумевано користи базе података засноване на датотекама (LanceDB за векторе, Kuzu за граф базе података).

    - **Претраживање свесно односа**: Одговори могу бити засновани не само на „сличном тексту“, већ и на томе како су ентитети повезани.

    - **Жива меморија**: Слој меморије се развија, расте и остаје претражив као један повезан граф.

2. Модови претраге и резоновања
    - **Хибридно претраживање**: Претрага комбинује сличност вектора, структуру графа и резоновање LLM-а, од основног проналажења делова текста до одговарања на питања свесних графа.

    - **Прилагодите мод задатку**: Користите модове у стилу комплетирања када желите одговоре на природном језику, а модове за делове/резиме/граф када вашем агенту треба сирови контекст или да води сопствено резоновање.

3. Персонализовани агенти свесни сесије
    - **Контекст сесије + дугорочна меморија**: Cognee одваја краткорочни контекст „теме“ од дугорочне меморије на нивоу корисника или организације.

## Примена у стварном свету

1. **Вертикални AI агенти**

    Користите образац из овог нотебука за покретање доменски паметних копилота који се ослањају на Cognee као језгро за претраживање и резоновање:

- **Копилоти за програмере**: Асистенти за преглед кода, анализу инцидената и архитектуру који прелазе кроз код, API-је, дизајнерске документе и тикете као један граф меморије.

- **Копилоти за кориснике**: Агенти за подршку или успех који извлаче информације из документације о производу, FAQ-ова, CRM белешки и претходних тикета уз претраживање свесно графа и одговоре са навођењем извора.

- **Интерни експертски копилоти**: Асистенти за политику, правна питања или безбедност који резонују преко међусобно повезаних правила, смерница и историјских одлука уместо изолованих PDF-ова.

    Cognee је експлицитно позициониран као трајна, прецизна меморија за AI агенте, пружајући живи граф знања који се уклапа иза вашег агента и замењује ад-хок комбинације складишта вектора и прилагођеног граф кода.

2. **Уједињавање податковних силоса у једну меморију**

    Исти приступ вам такође помаже да изградите уједињени слој меморије преко разбацаних извора:

- **Од силоса до једног графа**: Унесите структуиране (нпр. базе података) и неструктуиране податке (нпр. документе, четове) у један граф подржан уграђивањима, уместо одвојених индекса за сваки систем.

- **Резоновање преко извора са навођењем извора**: Спроведите вишестепено резоновање преко свега—„спојите“ логове, метрике и документе преко графа—и ипак вратите одговоре са доказима и пореклом.

- **Чворишта знања**: За домене као што су банкарство или образовање, Cognee се већ користи за уједињавање PDF-ова, интерних система и података апликација у један граф знања са векторима, тако да агенти могу одговарати на питања са прецизним, наведеним контекстом.

## Следећи кораци

Имплементирали сте основну петљу меморије. Ево природних проширења која можете испробати сами (погледајте [Cognee документацију](https://docs.cognee.ai/) за детаље):

1. **Експериментишите са временском свешћу**: Укључите временску когнификацију да извучете догађаје и временске ознаке из текста.

2. **Уведите резоновање вођено онтологијом**: Дефинишите OWL онтологију за ваш домен. Користите Cognee-ову подршку за онтологије како би извучени ентитети и односи били засновани на тој шеми, побољшавајући квалитет графа и одговоре специфичне за домен.

3. **Додајте петљу повратних информација**: Дозволите Cognee-у да прилагођава тежине ивица графа на основу стварних повратних информација корисника, тако да се претраживање временом побољшава уместо да остаје статично.

4. **Прилагодите за персонализацију и понашање сесије**: Користите ID-ове корисника, закупце и скупове података да свакој особи или тиму дате сопствени поглед на заједнички мотор меморије.

5. **Проширите на сложеније агенте**: Повежите Cognee са оквирима за агенте како бисте изградили системе са више агената који сви деле исти слој меморије. *Microsoft Agent Framework x Cognee додатак долази ускоро.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако тежимо тачности, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
